In [13]:
import torch
import os

def load_txt_as_ground_truth(txt_path):
    boxes = []
    labels = []

    if os.path.exists(txt_path):
        with open(txt_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) != 5:
                    continue
                cls, x1, y1, x2, y2 = map(float, parts)
                labels.append(int(cls))
                boxes.append([x1, y1, x2, y2])

    if boxes:
        return {
            "boxes": torch.tensor(boxes, dtype=torch.float32),
            "labels": torch.tensor(labels, dtype=torch.int64)
        }
    else:
        # Return empty tensors if no annotations or file doesn't exist
        return {
            "boxes": torch.zeros((0, 4), dtype=torch.float32),
            "labels": torch.zeros((0,), dtype=torch.int64)
        }

# Example: load GTs for a list of images
img_dir = "/home/umang.shikarvar/CycleGAN/lucknow_airshed/test/images"
label_dir = "/home/umang.shikarvar/CycleGAN/lucknow_airshed/test/labels"  # .txt files with same name as image but .txt extension

image_filenames = sorted([
    f for f in os.listdir(img_dir)
    if f.lower().endswith('.tif')
])

gt_targets = []

for img_file in image_filenames:
    base_name = os.path.splitext(img_file)[0]
    txt_file = os.path.join(label_dir, base_name + ".txt")
    gt = load_txt_as_ground_truth(txt_file)
    gt_targets.append(gt)

In [14]:
print(gt_targets)

[{'boxes': tensor([], size=(0, 4)), 'labels': tensor([], dtype=torch.int64)}, {'boxes': tensor([], size=(0, 4)), 'labels': tensor([], dtype=torch.int64)}, {'boxes': tensor([], size=(0, 4)), 'labels': tensor([], dtype=torch.int64)}, {'boxes': tensor([], size=(0, 4)), 'labels': tensor([], dtype=torch.int64)}, {'boxes': tensor([], size=(0, 4)), 'labels': tensor([], dtype=torch.int64)}, {'boxes': tensor([], size=(0, 4)), 'labels': tensor([], dtype=torch.int64)}, {'boxes': tensor([], size=(0, 4)), 'labels': tensor([], dtype=torch.int64)}, {'boxes': tensor([], size=(0, 4)), 'labels': tensor([], dtype=torch.int64)}, {'boxes': tensor([], size=(0, 4)), 'labels': tensor([], dtype=torch.int64)}, {'boxes': tensor([], size=(0, 4)), 'labels': tensor([], dtype=torch.int64)}, {'boxes': tensor([], size=(0, 4)), 'labels': tensor([], dtype=torch.int64)}, {'boxes': tensor([], size=(0, 4)), 'labels': tensor([], dtype=torch.int64)}, {'boxes': tensor([], size=(0, 4)), 'labels': tensor([], dtype=torch.int64)}

In [15]:
def load_txt_as_prediction(txt_path):
    boxes = []
    labels = []
    scores = []

    if os.path.exists(txt_path):
        with open(txt_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) != 6:
                    continue
                cls, conf, x1, y1, x2, y2 = map(float, parts)
                labels.append(int(cls))
                scores.append(conf)
                boxes.append([x1, y1, x2, y2])

    if boxes:
        return {
            "boxes": torch.tensor(boxes, dtype=torch.float32),
            "labels": torch.tensor(labels, dtype=torch.int64),
            "scores": torch.tensor(scores, dtype=torch.float32)
        }
    else:
        # Return empty prediction if no data
        return {
            "boxes": torch.zeros((0, 4), dtype=torch.float32),
            "labels": torch.zeros((0,), dtype=torch.int64),
            "scores": torch.zeros((0,), dtype=torch.float32)
        }

In [16]:
prediction_dir = "/home/umang.shikarvar/CycleGAN/lucknow_predictions"

predictions = []

for img_file in image_filenames:
    base_name = os.path.splitext(img_file)[0]
    pred_file = os.path.join(prediction_dir, base_name + ".txt")
    pred = load_txt_as_prediction(pred_file)
    predictions.append(pred)

In [17]:
def make_class_agnostic(detection_list):
    """
    Sets all labels in each detection dict to class 0
    """
    for d in detection_list:
        d["labels"] = torch.zeros_like(d["labels"])
    return detection_list

In [19]:
from torchmetrics.detection.mean_ap import MeanAveragePrecision

# Class-agnostic conversion
gt_targets_ca = make_class_agnostic(gt_targets)
predictions_ca = make_class_agnostic(predictions)

# Compute mAP
metric = MeanAveragePrecision()
metric.update(predictions_ca, gt_targets_ca)
results = metric.compute()

print("Class-Agnostic mAP@0.50:", results["map_50"].item())

Class-Agnostic mAP@0.50: 0.7663448452949524
